In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import math
import pickle
import random
import statistics
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#from dataprep.eda import plot
#import cx_Oracle


import warnings


import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler


import lightgbm as lgb
from matplotlib import font_manager, rc
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import pairwise_distances  #cosine 유사도 구할때 사용

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, precision_score, recall_score,classification_report,f1_score,roc_auc_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances  #cosine 유사도 구할때 사용

import surprise
from surprise import SVD
from surprise.model_selection import GridSearchCV,RandomizedSearchCV
from surprise import CoClustering
from surprise import SlopeOne
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split as train_test_split_su
from surprise import BaselineOnly
from surprise import NMF
from surprise.model_selection import KFold

from collections import defaultdict


import warnings
warnings.filterwarnings('ignore')

# set option
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)
pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:.4f}'.format

plt.rc("font", family="Malgun Gothic") # 한글깨짐현상 방지 

### 3.2.4 cosine 유사도 - 사용자간 유사도

##### 3.2.4.1 사용자 유사도를 위해 데이터 분포 파악

In [ ]:
#data_20220422 df_add.to_csv('data_20220422/data.csv', index=False, encoding = 'utf8') 
df_add = pd.read_csv('data.csv',encoding = 'utf-8')

In [ ]:
df_add.head(2)

In [ ]:
df_add['최종학력코드'].value_counts()

In [ ]:
df_add.loc[(df_add['나이'] >= 20) & (df_add['나이'] < 30) , '연령대'] = 20
df_add.loc[(df_add['나이'] >= 30) & (df_add['나이'] < 40) , '연령대'] = 30
df_add.loc[(df_add['나이'] >= 40) & (df_add['나이'] < 50) , '연령대'] = 40
df_add.loc[(df_add['나이'] >= 50) & (df_add['나이'] < 60) , '연령대'] = 50
df_add.loc[(df_add['나이'] >= 60) & (df_add['나이'] < 70) , '연령대'] = 60
df_add.loc[(df_add['나이'] >= 10) & (df_add['나이'] < 20) , '연령대'] = 10
df_add.loc[(df_add['나이'] < 10) , '연령대'] = 0
df_add.loc[(df_add['나이'] >= 70) ,'연령대'] = 70

In [ ]:
df_add.head(2)

In [ ]:
# 중요 FEATURE 추출
df_cos = df_add[['고객번호','성별구분','연령대','나이','직업위험등급','직업대분류코드'
                 ,'배우자여부','자녀여부','설계사수' ,'경제활동참가율'#'지점수','설계사수'
                 ,'스타벅스매장수','2020_건강생활실천율','시도명','상품분류코드_1'#,'유지계약건수','성립계약건수'
                 #'상급종합병원','2020_건강생활실천율','2020_출생률' ,'평균연령'
                 # 'FP몰선호상품코드','모집대면비대면여부',
                 #,'2020_노인여가복지시설수' ,'학문/교육' 
                ]] 

In [ ]:
df_cos_label = df_cos.copy()

In [ ]:
# 직업대분류 코드 LABEL 화 
df_cos_label = pd.get_dummies(df_cos_label, columns = ['직업대분류코드'])

In [ ]:
df_cos = df_cos_label.copy()

In [ ]:
# temp DF생성 
df_add_temp = df_add[['고객번호','상품분류코드_1']]

In [ ]:
df_add_temp

In [ ]:
df_add_cus_index = df_add_temp.set_index('고객번호')

In [ ]:
df2 = df_add_temp.copy()

In [ ]:
df2

In [ ]:
df_add_cus_index

### 최근 많이 판매된 상품 top3 구하기

In [ ]:
df_add.head(2)

In [ ]:
fil_sal = (df_add.계약월_1 >= 202106)   #data_final
df_add_top_sal = df_add.loc[fil_sal,:]  

In [ ]:
df_add_top_sal.groupby('계약월_1').sum()

In [ ]:
df_top_sal = df_add_top_sal['상품분류코드_1']

In [ ]:
df_top_sal

In [ ]:
top_sal_list  = []
top_sal_list = df_top_sal.value_counts().index[:3]
top_sal_list

In [ ]:
df_top_sal.value_counts()

##### knn data read

In [ ]:
# 중요 FEATURE 추출
df_knn = df_add[['고객번호','성별구분','나이','직업위험등급','직업대분류코드'
                 ,'배우자여부','자녀여부','설계사수','보건진료소' #'지점수','설계사수'
                 ,'스타벅스매장수','2020_건강생활실천율','시도명', '경제활동참가율'
                 #'상급종합병원','2020_건강생활실천율','2020_출생률' # 경제활동참가율
                 # 'FP몰선호상품코드','모집대면비대면여부',,'평균연령'
                 #,'2020_노인여가복지시설수' ,'학문/교육' 
                ]] 

In [ ]:
# 직업대분류 코드 LABEL 화 
df_knn = pd.get_dummies(df_knn, columns = ['직업대분류코드'])

In [ ]:
# 시도명 label화
df_knn = pd.get_dummies(df_knn, columns = ['시도명'])

In [ ]:
df_knn_copy = df_knn[['고객번호']]

In [ ]:
df_knn_copy.to_csv('df_knn_copy_random.csv',index = False)

In [ ]:
df_knn = df_knn.drop(['고객번호'],axis = 1)

In [ ]:
# 정규화 진행
scaler = MinMaxScaler()
data_scale = scaler.fit_transform(df_knn)

In [ ]:
# 그룹수를 50개 정도로 조정 
k = 50

# 그룹 수, random_state 설정
model = KMeans(init = 'random' , n_clusters = k ) #init = 'k-means++'random random_state = 10

# 정규화된 데이터에 학습
model.fit(data_scale)

# 클러스터링 결과 각 데이터가 몇 번째 그룹에 속하는지 저장
df_knn['cluster'] = model.fit_predict(data_scale)

In [ ]:
df_knn

### data 추출

In [ ]:
def make_data (df_cos, option ,sex,age,addr,k,size):
    if option == 'human':
        # 들어온 값을 시도 한글로 변경 
        if addr == 0 :
            addr = '서울특별시' 
        elif addr == 1 :
            addr = '부산광역시'         
        elif addr == 2 :
            addr = '대구광역시'
        elif addr == 3 :
            addr = '인천광역시'
        elif addr == 4 :
            addr = '광주광역시'
        elif addr == 5 :
            addr = '대전광역시'
        elif addr == 6 :
            addr = '울산광역시'
        elif addr == 7 :
            addr = '세종특별자치시'
        elif addr == 8 :
            addr = '경기도'
        elif addr == 9 :
            addr = '강원도'
        elif addr == 10 :
            addr = '충청북도'            
        elif addr == 11 :
            addr = '충청남도'  
        elif addr == 12 :
            addr = '전라북도'              
        elif addr == 13 :
            addr = '전라남도'             
        elif addr == 14 :
            addr = '경상북도'              
        elif addr == 15 :
            addr = '경상남도'             
        elif addr == 16 :
            addr = '제주특별자치도'
        else:
            print('시도 error : ', addr)
          
        print('성별: {0}, 연령대 : {1}, 시도명 : {2}'.format(sex,age,addr))
        #fil_yn = '##'
        #if age >= 60:
        #    fil_yn = (df_cos.성별구분 == sex) & (df_cos.연령대 >= age)  & (df_cos.시도명 == addr)  
        #else: 
        fil_yn = (df_cos.성별구분 == sex) & (df_cos.연령대 == age)  & (df_cos.시도명 == addr)  #data_final
        filename = 'df_cos_{0}_{1}_{2}'.format(sex,age,addr)
       # print(filename)
        data = df_cos.loc[fil_yn,:]
        data = data.drop(['성별구분','연령대','시도명'],axis = 1 )
    elif option == 'knn':
        fil_yn = (df_knn.cluster == k)   #data_final
        filename = df_knn.loc[fil_yn,:]   
        data = pd.merge(left = filename , right = df_knn_copy, left_index=True, right_index=True, how='left')
        data = data.drop(['cluster'],axis=1)
    else :
        data = pd.get_dummies(df_cos, columns = ['시도명'])
        data = data.sample(n=size,replace=False) # replace=True 비복원추출 
    #print("data.shape",data.shape)    
    return data    
        

In [ ]:
def cos_reco(data):
    #print("data_shape",data.shape)
    #----------cosine 유사도를 구하기 위해 StandardScaler start -----------------------
    data = data.set_index('고객번호',drop=True)
    print("StandardScaler start! ")
    # StandardScaler 사용 
    # 객체생성
    scaler = StandardScaler()

    # fit -> transform
    scaler.fit(data) # df는 2차원 이상의 값이어야 함
    df_scaled = scaler.transform(data)

    # 배열형태로 반환되기 때문에 데이터 프레임으로 변환해주는 작업
    df_scaled = pd.DataFrame(data = df_scaled, index=data.index, columns=data.columns)

    # *** 만약 특정 열의 스케일링을 하고 싶은 경우 ***
    #A_n = scaler.fit_transform(df['A'].values.reshape(-1,1))
    #df.insert(0, 'A_scaled', A_n)
    #df.drop(['A'], axis=1, inplace=True)
    
    # 파일 copy 
    df = df_scaled.copy()
    print("StandardScaler End! ")
    #----------cosine 유사도를 구하기 위해 StandardScaler end -----------------------
    
    
    
    #----------cosine 유사도 start --------------------------------------------------
    print("cosine_similarity start! ")
    item_sim = cosine_similarity(df, df)

    # cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
    item_sim_df = pd.DataFrame(data=item_sim, index=df.index,
                           columns=df.index)
    #print(item_sim_df.shape)
    item_sim_df.head(10)
    
    df = item_sim_df.copy()
    print("cosine_similarity End! ")
    #----------cosine 유사도 end----0-------------------------------------------------
    
    #----------유사한 고객 산출 start--------------------------------------------------
    print("cosine_customer start! ")
    # 30개 추출 
    dic = {}
    for 고객번호 in df.columns:
        유사집단 = df.loc[int(고객번호), df.columns != 고객번호].sort_values(
            ascending=False).head(30).index.tolist()
        dic[고객번호]=유사집단
        
    df_tec = pd.DataFrame(dic).transpose()    
    #print('df_tec.shape',df_tec.shape)
    df = df_tec.copy()
    print("cosine_customer end! ")
    #----------유사한 고객 산출 end---------------------------------------------------

    #----------유사한 고객의 상품코드 추출 start -------------------------------------
    print("cosine_prod start! ")
    df = df.iloc[:,:]
    dic = {}
    keys = df.columns.tolist()
    for key in keys:
        dic[key] = df[key].tolist()
    
    고객번호_dic = {}
    고객번호_list = []
    keys = df.columns.tolist()
    #for index, 고객번호 in enumerate(df['고객번호'].tolist()):
    for index, 고객번호 in enumerate(df.index.tolist()):
        고객번호_dic[고객번호] = df.iloc[index].tolist()
    
    recommendation_dic = {}
    for key in 고객번호_dic.keys():
        #print(f'고객번호 : {key}')
        recommendation_list = []
        for index in range(len(고객번호_dic[key])):
            recommendation = df2.loc[df2['고객번호']==고객번호_dic[key][index], '상품분류코드_1'].values[0]
            recommendation_list.append(recommendation)
        recommendation_dic[key] = recommendation_list
   # print('df_rec.shape',df_rec.shape)
    pd.DataFrame(recommendation_dic).transpose()
    df_rec = pd.DataFrame(recommendation_dic).transpose()
    print("cosine_prod end! ")
    #----------유사한 고객의 상품코드 추출 end ----------------------------------------
    
    
    #----------유사한 고객의 상품코드 최빈값 start -------------------------------------
    print("customer mode start! ")
    # mode 최빈값 여러건인지 확인하기 
    df_rec['추천코드_1'] = '0'
    df_rec['추천코드_2'] = '0'
    df_rec['추천코드_3'] = '0'
    df_rec['추천코드_top'] = '0'
    # top_sal_list
    k = 1 
    for i in range (0,len(df_rec)):
        list = []
        list = str(statistics.mode(df_rec.iloc[i]))
        #print(list)
        
        df_rec["추천코드_1"].iloc[i] = str(statistics.mode(df_rec.iloc[i]))
        #print(str(statistics.mode(df_rec.iloc[i])))
        # 두번째 추천코드 
        # 두번째 추천코드
        if str(statistics.mode(df_rec.iloc[i])) != '0' :
            fil = (statistics.mode(df_rec.drop(['추천코드_1','추천코드_2','추천코드_3','추천코드_top'],axis=1).iloc[i]) 
                   != df_rec.drop(['추천코드_1','추천코드_2','추천코드_3','추천코드_top'],axis=1).iloc[i])
            prd_2 = df_rec.drop(['추천코드_1','추천코드_2','추천코드_3','추천코드_top'],axis=1).iloc[i][fil]
            if len(prd_2) != 0:
                df_rec["추천코드_2"].iloc[i]= str(statistics.mode(prd_2))
                # -----3번쨰 상품 추천을 위해 
                fil3 = ((statistics.mode(df_rec.drop(['추천코드_1','추천코드_2','추천코드_3','추천코드_top'],axis=1).iloc[i]) 
                        != df_rec.drop(['추천코드_1','추천코드_2','추천코드_3','추천코드_top'],axis=1).iloc[i]))
                fil4 = statistics.mode(prd_2) != df_rec.drop(['추천코드_1','추천코드_2','추천코드_3','추천코드_top'],axis=1).iloc[i] 
            
                fil_3 = ( fil3 == fil4)  
                prd_3 =  df_rec.drop(['추천코드_1','추천코드_2','추천코드_3','추천코드_top'],axis=1).iloc[i][fil_3]
                
                if len(prd_3)!= 0:
                    df_rec["추천코드_3"].iloc[i]= str(statistics.mode(prd_3))
                    #print(str(statistics.mode(prd_3)))
                else:
                    df_rec["추천코드_3"].iloc[i] = 0
                
            else :
                df_rec["추천코드_2"].iloc[i] = 0
        else :
            df_rec["추천코드_2"].iloc[i] = 0
        

        for u in range (0,3):
                     
            if (df_rec["추천코드_1"].iloc[i] != str(top_sal_list[u])) and (df_rec["추천코드_2"].iloc[i] != str(top_sal_list[u])):
                df_rec['추천코드_top'].iloc[i] =  str(top_sal_list[u]) 
                break
                
        
        #if str(statistics.mode(df_rec.iloc[i])) != '0' :
        #    #print("aaa")
        #    fil = (statistics.mode(df_rec.iloc[i]) != df_rec.iloc[i])
        #    prd_2 = df_rec.iloc[i][fil]
        #    df_rec["추천코드_2"].iloc[i]= str(statistics.mode(prd_2))
        #else :
        #    df_rec["추천코드_2"].iloc[i] = 0
        #    
        if len(list) > 3 :
            print("2개이상" , i)
            k = k + 1
        i = i+1
    #print(k)
    print("customer mode end! ")
    #----------유사한 고객의 상품코드 최빈값 end -------------------------------------
    
    #-----------추천 상품에 대한 정확도 측정 start ------------------------------------
   # print("correct start! ")
    
    df_t = pd.merge(left = df_rec , right = df_add_cus_index, left_index=True, right_index=True, how='left')
    
    df_t['일치여부_1'] = 0
    df_t['일치여부_2'] = 0
    df_t['일치여부_3'] = 0   
    df_t['일치여부_top'] = 0
    match1_y = 0
    match2_y = 0
    match3_y = 0
    match_top_y = 0 
    for i in range (0,len(df_t)): #len(df_t)

        if str(df_t['추천코드_1'].values[i]) == str(df_t['상품분류코드_1'].values[i]):
            df_t['일치여부_1'].iloc[i] = 1
            match1_y = match1_y+1
        else:
            df_t['일치여부_1'].iloc[i] = 0        
        
        if str(df_t['추천코드_2'].values[i]) == str(df_t['상품분류코드_1'].values[i]):
            df_t['일치여부_2'].iloc[i] = 1
            match2_y = match2_y + 1
        else:
            df_t['일치여부_2'].iloc[i] = 0      
        
        if str(df_t['추천코드_3'].values[i]) == str(df_t['상품분류코드_1'].values[i]):
            df_t['일치여부_3'].iloc[i] = 1
            match3_y = match3_y + 1
        else:
            df_t['일치여부_3'].iloc[i] = 0   
            
        if str(df_t['추천코드_top'].values[i]) == str(df_t['상품분류코드_1'].values[i]):
            df_t['일치여부_top'].iloc[i] = 1
            match_top_y = match_top_y + 1
        else:
            df_t['일치여부_top'].iloc[i] = 0              
    #print(match1_y/len(df_t), '/',match1_y, len(df_t))
    #print(match2_y/len(df_t), '/',match2_y, len(df_t))
    #print("correct end! ")
    return match1_y, match2_y,match3_y,match_top_y, len(df_t)
    #return df_t 
    
    #-----------추천 상품에 대한 정확도 측정 end ------------------------------------
    

## 모델별 정확도 측정

In [ ]:
df_knn_result = pd.DataFrame(columns=['idx', 'knn_cluster','match1_y','match2_y','match3_y','match_top_y','df_len','corr_rate'])

In [ ]:
df_knn_result

###### k-nn 클러스터별 정확도 측정

In [ ]:
for i in range (0,50):
    corr_rate = 0 
    data = make_data(df_knn,'knn',0 ,0 ,0,i,0) ## 앞에 어느 값을 줄지 선택 
    match1_y, match2_y, match3_y,match_top_y, df_len = cos_reco(data) 
    knn_cluster = i 
    idx = i 
    corr_rate  = round((int(match1_y)+int(match2_y)+int(match3_y)) / int(df_len), 6) 
    df_knn_result = df_knn_result.append(pd.DataFrame([[idx, knn_cluster,match1_y,match2_y,match3_y,match_top_y,df_len,corr_rate]],
                                   columns=['idx', 'knn_cluster','match1_y','match2_y','match3_y','match_top_y','df_len','corr_rate']), 
                                   ignore_index=True)
    #print('i',i)

In [ ]:
df_knn_result.head(2)

In [ ]:
df_knn_result.to_csv('df_random_20220520_v3.csv',index=False)

#### 사람이 정한 클러스터로 진행

In [ ]:
df_h = pd.DataFrame(columns=['idx', 'sex','years','region','match1_y','match2_y','match3_y','match_top_y','df_len','corr_rate'])

In [ ]:
for j in range (1,3): # 성별 (1,2) # 3
    corr_rate = 0 
    for k in range (1,7): # 8 --> 연령대 
        for u in range (0,17): # 18  --> 시도 
            data = make_data(df_cos,'human',j ,k*10 ,u,0,0) ## 앞에 어느 값을 줄지 선택  data = make_data(df_cos,'human',1,30,'서울특별시',1)
            if len(data) != 0 :
                # 유사도 측정 
                match1_y, match2_y, match3_y,match_top_y, df_len = cos_reco(data)  
                # 시도 setting 
                idx = (j + k + u )
                sex = j
                years = (k*10)
                region = u
                # 정확도 측정 
                corr_rate  = round((int(match1_y)+int(match2_y)+int(match3_y)) / int(df_len), 6) 
                df_h = df_h.append(pd.DataFrame([[idx, sex,years,region,match1_y,match2_y,match3_y,match_top_y,df_len,corr_rate]],
                                            columns=['idx', 'sex','years','region','match1_y','match2_y','match3_y','match_top_y','df_len','corr_rate']), 
                                            ignore_index=True)
            else:
                print("pass")
    

In [ ]:
df_h

In [ ]:
df_h.to_csv('df_h_20220521.csv',index=False)

# Model기반 추천 시스템

In [ ]:
df_add.shape

In [ ]:
# 중요 FEATURE 추출
df_DT  = df_add[['고객번호','성별구분','나이','직업위험등급','직업대분류코드'
                 ,'배우자여부','자녀여부','설계사수','보건진료소' #'지점수','설계사수'
                 ,'스타벅스매장수','2020_건강생활실천율','시도명', '경제활동참가율'
                 ,'상품분류코드_1'
                 #'상급종합병원','2020_건강생활실천율','2020_출생률' ,,'평균연령'
                 # 'FP몰선호상품코드','모집대면비대면여부',
                 #,'2020_노인여가복지시설수' ,'학문/교육' 
                ]] 

In [ ]:
# 직업대분류 코드 LABEL 화 
df_DT = pd.get_dummies(df_DT, columns = ['직업대분류코드'])
# 시도명 label화
df_DT = pd.get_dummies(df_DT, columns = ['시도명'])

In [ ]:
df_DT.info()

In [ ]:
df = df_DT.copy()

In [ ]:
df.head(1)

In [ ]:
df_DT

In [ ]:
df_x = df.drop(['상품분류코드_1'],axis = 1)

In [ ]:
df_x.shape

In [ ]:
df_y = df['상품분류코드_1']

In [ ]:
# 인덱스를 고객번호로
df = df.set_index('고객번호')
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(df_x, df_y, test_size=0.3)

### hyper parameter

In [ ]:
import optuna
import sklearn
def objective(trial):
    # Define the search space
    criterions = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depths = trial.suggest_int('max_depth', 1, 9, 1)
   # n_estimators = trial.suggest_int('n_estimators', 1, 20, 1)
    max_leaf_nodes = int(trial.suggest_int("max_leaf_nodes", 2, 50))

    clf = DecisionTreeClassifier(#n_estimators=n_estimators,
                                 max_leaf_nodes = max_leaf_nodes,                 
                                 criterion=criterions,
                                 max_depth=max_depths)
                               #  n_jobs=-1)
                                 
    score = cross_val_score(clf, x_train, y_train, scoring="accuracy").mean()

    return score

# 3. study 오브젝트 생성하고 목적함수 최적화하는 단계
# 여기서는 목적함수를 정확도로 설정했기 때문에 최대화를 목표로 하고 있지만, 손실함수의 경우 direction='minimize'로 설정
study = optuna.create_study()
study.optimize(objective, n_trials=50)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
# 시행된 trial 중 최적의 하이퍼파라미터 반환하는 메소드
print(study.best_trial.params)

# 시행된 trial 중 가장 높은 값 반환하는 메소드
optuna_acc = study.best_trial.value
print(optuna_acc)

### DecisionTreeClassifier

In [ ]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=6, max_leaf_nodes = 36, splitter='best', criterion='entropy')
dtc.fit(x_train, y_train)
dtc_pred = dtc.predict(x_valid)
(dtc_pred == y_valid).mean()

In [ ]:
dtc_pred


In [ ]:
# df for valid(test)
df_valid = x_valid 

In [ ]:
# 상품분류코드 idx
dtc.classes_

In [ ]:
item_code = {0: 111,
             1: 120,
             2: 130,
             3: 141,
             4: 142,
             5: 150,
             6: 162,
             7: 163,
             8: 164,
             9: 181,
             10: 182,
             11: 183,
             12: 192}


In [ ]:
# predict_proba 분류기 확률값 치환
dtc_pred_proba = dtc.predict_proba(x_valid)
dtc_pred_proba

In [ ]:
# 검증, 첫번째 분류기 output값이 150이 맞는지 확인해보자
a = dtc_pred_proba[0]

# np.argpartition(sth, 2) -> 순서 상관 없이 2번째까지 작은 숫자 idx 뽑아서 왼쪽으로 놓는다
# np.argpartition(sth, -2) -> 순서 상관 없이 2번째까지 큰 숫자 idx 뽑아서 오른쪽으로 놓는다

# 2번째까지 큰 숫자 idx 뽑아서 오른쪽으로 놓고
# 오른쪽 2개를 idx로 가져온다
idx = np.argpartition(a, -3)[-3:] # idx = [rank2, rank1]

# idx는 상품분류코드 번호를 뜻하게 된다
# reverse -> [rank1, rank2]
idx = np.sort(idx)[::-1]
idx

In [ ]:
# df_valid(test) 생성
df_valid = x_valid.copy()

# rank 컬럼 생성
df_valid['rank1']=''
df_valid['rank2']=''
df_valid['rank3']=''

In [ ]:
for index, 고객번호 in enumerate(df_valid.index):
    추천상품 = np.argpartition(dtc_pred_proba[index], -3)[-3:]
    df_valid.loc[고객번호, 'rank1'] = 추천상품[0]
    df_valid.loc[고객번호, 'rank2'] = 추천상품[1]
    df_valid.loc[고객번호, 'rank3'] = 추천상품[2]


In [ ]:
# 상품분류코드로 변환하기
df_valid['rank1'] = df_valid['rank1'].replace(item_code)
df_valid['rank2'] = df_valid['rank2'].replace(item_code)
df_valid['rank3'] = df_valid['rank3'].replace(item_code)


In [ ]:
df_valid.head(2)

In [ ]:
# 30% 테스트 
df_valid_DT.shape

In [ ]:
df_DT_cus = df_add[['고객번호','상품분류코드_1']]

In [ ]:
df_DT_cus.shape

In [ ]:
df_DT_cus = df_DT_cus.set_index('고객번호')

In [ ]:
df_valid_DT = df_valid.set_index('고객번호')

In [ ]:
df_DT_cus.head(1)

In [ ]:
df_valid_DT.head(2)

In [ ]:
df_t = pd.merge(left = df_valid_DT , right = df_DT_cus, left_index=True, right_index=True, how='left')

In [ ]:
df_t.head(1)

In [ ]:
df_t['일치여부_1'] = 0
df_t['일치여부_2'] = 0
df_t['일치여부_3'] = 0
match1_y = 0
match2_y = 0
match3_y = 0
for i in range (0,len(df_t)): #len(df_t)

    if str(df_t['rank1'].values[i]) == str(df_t['상품분류코드_1'].values[i]):
        df_t['일치여부_1'].iloc[i] = 1
        match1_y = match1_y+1
    else:
        df_t['일치여부_1'].iloc[i] = 0        
    
    if str(df_t['rank2'].values[i]) == str(df_t['상품분류코드_1'].values[i]):
        df_t['일치여부_2'].iloc[i] = 1
        match2_y = match2_y + 1
    else:
        df_t['일치여부_2'].iloc[i] = 0 

    if str(df_t['rank3'].values[i]) == str(df_t['상품분류코드_1'].values[i]):
        df_t['일치여부_3'].iloc[i] = 1
        match3_y = match3_y + 1
    else:
        df_t['일치여부_3'].iloc[i] = 0         

In [ ]:
df_t.head(10)

In [ ]:
match1_y, match2_y, len(df_t)  

In [ ]:
(match1_y + match2_y) / len(df_t)  

# Random Forest

In [ ]:
x_train

In [ ]:
y_train

### hyperparamete 

In [ ]:
import optuna
import sklearn
def objective(trial):
    # Define the search space
    criterions = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth', 1, 10)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 1000)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)

    clf = sklearn.ensemble.RandomForestClassifier( criterion=criterions,
                                 n_estimators=n_estimators,
                                 max_leaf_nodes = max_leaf_nodes,                 
                                 max_depth=max_depth,
                                 n_jobs=-1,
                                 random_state=0                 
                                                 )
                                 
    score = cross_val_score(clf, x_train, y_train, scoring="accuracy").mean()

    return score

# 3. study 오브젝트 생성하고 목적함수 최적화하는 단계
# 여기서는 목적함수를 정확도로 설정했기 때문에 최대화를 목표로 하고 있지만, 손실함수의 경우 direction='minimize'로 설정
study = optuna.create_study()
study.optimize(objective, n_trials=50)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
# 시행된 trial 중 최적의 하이퍼파라미터 반환하는 메소드
print(study.best_trial.params)

# 시행된 trial 중 가장 높은 값 반환하는 메소드
optuna_acc = study.best_trial.value
print(optuna_acc)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=8,max_leaf_nodes = 733, n_estimators= 500, criterion='entropy', random_state=0) 
# (n_estimators=474, max_depth=1, max_leaf_nodes=552, random_state=25)
rfc.fit(x_train, y_train)
rfc_pred = rfc.predict(x_valid)
(rfc_pred == y_valid).mean()


In [ ]:
rfc.classes_

In [ ]:
# predict_proba 분류기 확률값 치환
rfc_pred_proba = rfc.predict_proba(x_valid)
rfc_pred_proba

In [ ]:
for index, 고객번호 in enumerate(df_valid.index):
    추천상품 = np.argpartition(rfc_pred_proba[index], -3)[-3:]
    df_valid.loc[고객번호, 'rank1'] = 추천상품[0]
    df_valid.loc[고객번호, 'rank2'] = 추천상품[1]
    df_valid.loc[고객번호, 'rank3'] = 추천상품[2]


In [ ]:
# 상품분류코드로 변환하기
df_valid['rank1'] = df_valid['rank1'].replace(item_code)
df_valid['rank2'] = df_valid['rank2'].replace(item_code)
df_valid['rank3'] = df_valid['rank3'].replace(item_code)

In [ ]:
df_valid

In [ ]:
df_valid.to_csv('df_valid_RF.csv',index = False)

In [ ]:
df_valid_RF = pd.read_csv('df_valid_RF.csv')

In [ ]:
df_valid_RF = df_valid.copy()

In [ ]:
df_valid_RF = df_valid_RF.set_index('고객번호')

In [ ]:
df_t1 = pd.merge(left = df_valid_RF , right = df_DT_cus, left_index=True, right_index=True, how='left')

In [ ]:
df_t1.shape

In [ ]:
df_t1.head(2)

In [ ]:
df_t1['일치여부_1'] = 0
df_t1['일치여부_2'] = 0
df_t1['일치여부_3'] = 0

match1_y = 0
match2_y = 0
match3_y = 0

for i in range (0,len(df_t1)): #len(df_t)

    if str(df_t1['rank1'].values[i]) == str(df_t1['상품분류코드_1'].values[i]):
        df_t1['일치여부_1'].iloc[i] = 1
        match1_y = match1_y+1
    else:
        df_t1['일치여부_1'].iloc[i] = 0        
    
    if str(df_t1['rank2'].values[i]) == str(df_t1['상품분류코드_1'].values[i]):
        df_t1['일치여부_2'].iloc[i] = 1
        match2_y = match2_y + 1
    else:
        df_t1['일치여부_2'].iloc[i] = 0 
        
    if str(df_t1['rank3'].values[i]) == str(df_t1['상품분류코드_1'].values[i]):
        df_t1['일치여부_3'].iloc[i] = 1
        match3_y = match3_y + 1
    else:
        df_t1['일치여부_3'].iloc[i] = 0         

In [ ]:
match1_y, match2_y, len(df_t)  

In [ ]:
(match1_y+ match2_y) / len(df_t)  

# SVD

### 3.1 협업필터링(사용자간유사도_사용자기반_상품가입건_유)

In [ ]:
df_sim = df_add[['고객번호','상품분류코드_1','상품분류코드_2','상품분류코드_3']]

In [ ]:
df_sim[:3]

In [ ]:
# 사용자간 선호도를 상품 종류 코드에 따라 가입건수 count로 설정 
df_sim = pd.get_dummies(df_sim, columns = ['상품분류코드_1'])
df_sim = pd.get_dummies(df_sim, columns = ['상품분류코드_2'])
df_sim = pd.get_dummies(df_sim, columns = ['상품분류코드_3'])

In [ ]:
df_sim.head(2)

In [ ]:
df_sim['상품분류코드_111'] = df_sim['상품분류코드_1_111'] + df_sim['상품분류코드_2_111.0'] + df_sim['상품분류코드_3_111.0']
df_sim['상품분류코드_120'] = df_sim['상품분류코드_1_120'] + df_sim['상품분류코드_2_120.0'] + df_sim['상품분류코드_3_120.0']
df_sim['상품분류코드_130'] = df_sim['상품분류코드_1_130'] + df_sim['상품분류코드_2_130.0'] + df_sim['상품분류코드_3_130.0']
df_sim['상품분류코드_141'] = df_sim['상품분류코드_1_141'] + df_sim['상품분류코드_2_141.0'] + df_sim['상품분류코드_3_141.0']
df_sim['상품분류코드_142'] = df_sim['상품분류코드_1_142'] + df_sim['상품분류코드_2_142.0'] + df_sim['상품분류코드_3_142.0']
df_sim['상품분류코드_150'] = df_sim['상품분류코드_1_150'] + df_sim['상품분류코드_2_150.0'] + df_sim['상품분류코드_3_150.0']
df_sim['상품분류코드_162'] = df_sim['상품분류코드_1_162'] + df_sim['상품분류코드_2_162.0'] + df_sim['상품분류코드_3_162.0']
df_sim['상품분류코드_163'] = df_sim['상품분류코드_1_163'] + df_sim['상품분류코드_2_163.0'] + df_sim['상품분류코드_3_163.0']
df_sim['상품분류코드_164'] = df_sim['상품분류코드_1_164'] + df_sim['상품분류코드_2_164.0'] + df_sim['상품분류코드_3_164.0']
df_sim['상품분류코드_181'] = df_sim['상품분류코드_1_181'] + df_sim['상품분류코드_2_181.0'] + df_sim['상품분류코드_3_181.0']
df_sim['상품분류코드_182'] = df_sim['상품분류코드_1_182'] + df_sim['상품분류코드_2_182.0'] 
df_sim['상품분류코드_183'] = df_sim['상품분류코드_1_183'] + df_sim['상품분류코드_2_183.0'] + df_sim['상품분류코드_3_183.0']
df_sim['상품분류코드_192'] = df_sim['상품분류코드_1_192'] + df_sim['상품분류코드_2_192.0'] + df_sim['상품분류코드_3_192.0']

In [ ]:
df_sim.head(2)

In [ ]:
df_sim = df_sim[['고객번호',
                 '상품분류코드_111',
                 '상품분류코드_120',
                 '상품분류코드_130',
                 '상품분류코드_141',
                 '상품분류코드_142',
                 '상품분류코드_150',
                 '상품분류코드_162',
                 '상품분류코드_163',
                 '상품분류코드_164',
                 '상품분류코드_181',
                 '상품분류코드_182',
                 '상품분류코드_183',
                 '상품분류코드_192' ]]

In [ ]:
df_sim = df_sim.rename(columns = {'상품분류코드_111':'111'})
df_sim = df_sim.rename(columns = {'상품분류코드_120':'120'})
df_sim = df_sim.rename(columns = {'상품분류코드_130':'130'})
df_sim = df_sim.rename(columns = {'상품분류코드_141':'141'})
df_sim = df_sim.rename(columns = {'상품분류코드_142':'142'})
df_sim = df_sim.rename(columns = {'상품분류코드_150':'150'})
df_sim = df_sim.rename(columns = {'상품분류코드_162':'162'})
df_sim = df_sim.rename(columns = {'상품분류코드_163':'163'})
df_sim = df_sim.rename(columns = {'상품분류코드_164':'164'})
df_sim = df_sim.rename(columns = {'상품분류코드_181':'181'})
df_sim = df_sim.rename(columns = {'상품분류코드_182':'182'})
df_sim = df_sim.rename(columns = {'상품분류코드_183':'183'})
df_sim = df_sim.rename(columns = {'상품분류코드_192':'192'})

In [ ]:
df1 = df_sim.melt(id_vars = "고객번호",var_name = "prodId",value_name = "rating")

In [ ]:
df1 = df1.rename(columns = {'고객번호':'userId'})

In [ ]:
df2 = df1.groupby('userId').sum().reset_index()

In [ ]:
df = pd.merge(df1, df2, how='inner', on='userId')

In [ ]:
df = df.rename(columns={'prodId_x': 'prodId', 'rating_x': 'rating_1', 'rating_y':'rating_2'})

In [ ]:
print("Total no of ratings :",df.shape[0])
print("No. of unique users:", df["userId"].nunique())
print("No. of unique movies:", df["prodId"].nunique())

In [ ]:
df[:13]

In [ ]:
from collections import defaultdict

from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=3, threshold=1.0):
    """Return precision and recall at k metrics for each user"""
    #print("predictions : ",predictions)
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, prod_cd, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
        
     #   print("input: ", uid,prod_cd,true_r,est,prod_cd)

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        #print("aa:",user_ratings.items)
        #print("user_ratings:",user_ratings)
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
        #print("user_ratings[:k]",user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


reader = Reader()
ratings = Dataset.load_from_df(df[['userId', 'prodId', 'rating_1']], reader)

kf = KFold(n_splits=2)
algo = SVD()

for trainset, testset in kf.split(ratings):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=3, threshold=1.0)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))